```bash
wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh

chmod +x Miniconda3-latest-Linux-x86_64.sh

./Miniconda3-latest-Linux-x86_64.sh

Please, press ENTER to continue
```

Einmal Leertaste drücken.

```bash
Please answer 'yes' or 'no':'
>>> yes

Press ENTER to confirm the location

Do you wish the installer to initialize Miniconda3
by running conda init? [yes|no]
[no] >>> yes

conda config --set auto_activate_base false

conda create -n python_workshop_env python=3

Proceed ([y]/n)? y

. ~/.bashrc

conda activate python_workshop_env

conda install s3fs bokeh dask ipykernel numpy pandas "dask[distributed]" mpi4py

Proceed ([y]/n)? y

conda install -c conda-forge dask-mpi
```
